## Парсинг новостных сайтов. XPath

В данном ноутбуке реализован парсинг новостей с сайтов:
- [Новости mail.ru](https://news.mail.ru/)
- [Яндекс новости](https://yandex.ru/news/)
- [Lenta.ru](https://lenta.ru/)

Результаты сохраняются в коллекцию базы news, содержащую следующие ключи:
- Id объекта
- Дата публикации
- Время публикации
- Заголовок новости
- Cсылка на новость
- Название источника
- Ссылка на источник (если ресурс предоставляет, имеется ссылка на саму новость в источнике)


Если какой-то из элементов не найден в поле записывается значение None. Исключение - дата и время - в случае их отсутствия ставится текущее время и дата.

In [1]:
import datetime
from lxml import html
import requests
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['news_parser_db']

In [3]:
news = db.news

In [4]:
months = {
'января': '01',
'февраля': '02',
'марта': '03',
'апреля': '04',
'мая': '05',
'июня': '06',
'июля': '07',
'августа': '08',
'сентября': '09',
'октября': '10',
'ноября': '11',
'декабря': '12',
}

mail_news = 'https://news.mail.ru'
yandex_news = 'https://yandex.ru/news'
lenta_ru = 'https://lenta.ru'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

mail_links = []
lenta_links = []
yandex_links = []

### Парсинг "Новости Mail.ru"

Принцип работы:
1. Получаем ссылку на главную новость (1 шт.), другие главные новости (4 шт. без адаптивной верстки), остальные новости (6 шт.) и получаем список mail_links.
2. Для каждого объекта в списке переходим по ссылке при помощи функции get_item_info_mail и получаем нужные поля.
3. Добавляем объекты в коллекцию news.

In [5]:
def get_item_info_mail(link):
    news_object = {}
    response = requests.get(link, headers=headers)
    if not response.ok:
        print(response.status_code)
        return
    
    root = html.fromstring(response.text)

    # ссылка
    news_object['url'] = link

    # заголовок
    title = root.xpath("//h1[@class = 'hdr__inner']/text()")
    if title:
        news_object['title'] = title[0]
    else:
        news_object['title'] = None

    # источник
    source_title = root.xpath("//span[@class = 'breadcrumbs__item']//a/span/text()")
    if source_title:
        news_object['source_title'] = source_title[0]
    else:
        news_object['source_title'] = None
    
    source_url = root.xpath("//span[@class = 'breadcrumbs__item']//a/@href")
    if source_url:
        news_object['source_url'] = source_url[0]

    # дата и время
    date_time = root.xpath("//span[@class = 'breadcrumbs__item']//span/@datetime")
    # если не получилось получить date_time, ставим текущие
    if not date_time:
        news_object['date'] = str(datetime.datetime.now().date())
        news_object['time'] = str(datetime.datetime.now().time())

    date_time_items = date_time[0].split('T')
    news_object['date'] = date_time_items[0]
    time = date_time_items[1].split('+')
    news_object['time'] = time[0]

    return news_object 

In [6]:
mail_page = requests.get(mail_news, headers=headers)

if not mail_page.ok:
    print(mail_page.status_code)
    
root = html.fromstring(mail_page.text)

# топ новость
daily_news1 = root.xpath("//div[@class='daynews__item daynews__item_big']/a/@href")[0]
# обработка случая, когда ссылка на другой ресурс mail (например, sportmail)
if 'http' in daily_news1:
    daily_news1 = daily_news1
else:
    daily_news1_url = mail_news + daily_news1
mail_links.append(daily_news1_url)


# топ других новостей
daily_news2 = root.xpath("//div[@class= 'daynews__item']/a/@href")
for item in daily_news2:
    if 'http' in item:
        daily_news2_url = item
    else:
        daily_news2_url = mail_news + item
    mail_links.append(daily_news2_url)


# другие новости
daily_news3 = root.xpath("//ul[@class = 'list list_type_square list_half js-module']/li/a/@href")
for item in daily_news3:
    if 'http' in item:
        daily_news3_url = item
    else:
        daily_news3_url = mail_news + item
    mail_links.append(daily_news3_url)


# собираем информацию об объектах 
for link in mail_links:
    news_obj = get_item_info_mail(link)

    # пополняем коллекцию
    obj = next(news.find({"url": {'$eq': news_obj['url']}}), None)
    if not obj:
        news.insert_one(news_obj)
    else:
        news.update_one({"id": {'$eq': news_obj['url']}},{'$set': news_obj})

In [7]:
db.news.count_documents({})

11

### Парсинг Lenta.ru

Принцип работы:
1. Получаем ссылку на главную новость (1 шт.) и другие главные новости (9 шт.) и получаем список lenta_links.
2. Для каждого объекта в списке переходим по ссылке при помощи функций get_item_info_lenta и get_item_info_moslenta и получаем нужные поля.
3. Добавляем объекты в коллекцию news.

In [8]:
def get_item_info_moslenta(link):
    news_object = {}
    response = requests.get(link, headers=headers)
    
    if not response.ok:
        print(response.status_code)
        return
    
    root = html.fromstring(response.text)

    # ссылка
    news_object['url'] = link

    # источник
    news_object['source_url'] = 'https://moslenta.ru/'
    news_object['source_title'] = 'Мослента'

    # заголовок
    title = root.xpath("//h1[@class = 'jsx-3827467167 jsx-3813442946 headline']/text()")
    if title:
        news_object['title'] = title[0]
    else:
        news_object['title'] = None

    # дата и время
    date_time = root.xpath("//div[@class = 'jsx-149585235 jsx-932351741 topline']//text()")
    # если не получилось получить date_time, ставим текующие
    if not date_time:
        news_object['date'] = str(datetime.datetime.now().date())
        news_object['time'] = str(datetime.datetime.now().time())
        
    date_time = date_time[0]
    date_time_items = date_time.split(' ')

    # время
    news_object['time'] = date_time_items[-1] + ':00'
    
    # дата
    if date_time_items[0] == 'Сегодня':
        news_object['date'] = str(datetime.date.today())
    elif date_time_items[0] == 'Вчера':
        news_obj['date'] = str(datetime.date.today() - datetime.timedelta(1))
    else: 
        day = date_time_items[0]
        month = months[date_time_items[1]]
        year = str(datetime.datetime.now().year)
        date = year + "-" + month + "-" + day
        news_object['date'] = date
    
    return news_object


def get_item_info_lenta(link):
    news_object = {}
    response = requests.get(link, headers=headers) 
    
    if not response.ok:
        print(response.status_code)
        return

    root = html.fromstring(response.text)

    # ссылка
    news_object['url'] = link

    # источник
    news_object['source_url'] = lenta_ru
    news_object['source_title'] = 'Lenta.ru'

    # заголовок
    title = root.xpath("//h1[@class ='b-topic__title']/text()")
    if title:
        news_object['title'] = title[0]
    else:
        news_object['title'] = None

    # дата и время
    date_time = root.xpath("//div[@class = 'b-topic__info']/time/@datetime")
    # если не получилось получить date_time, ставим текующие
    if not date_time:
        news_object['date'] = str(datetime.datetime.now().date())
        news_object['time'] = str(datetime.datetime.now().time())      

    date_time = date_time[0]
    date_time_items = date_time.split('T')

    news_object['date'] = date_time_items[0]

    time = date_time_items[1].split('+')
    news_object['time'] = time[0]

    return news_object

In [9]:
lenta_page = requests.get(lenta_ru, headers=headers)

if not lenta_page.ok: 
    print(lenta_page.status_code)
    
root = html.fromstring(lenta_page.text) 

# главный пост
lead_post = root.xpath("//div[@class = 'first-item']/a/@href")[0]
# обработка случая, когда ссылка на moslenta
if 'http' in lead_post:
    lead_post = lead_post
else:
    lead_post_url = lenta_ru + lead_post
    lenta_links.append(lead_post_url)

# основные новости
other_posts = root.xpath("//div[@class = 'span4']/div[@class = 'item']/a/@href")
for item in other_posts:
    if 'http' in item:
        other_post_url = item
    else:
        other_post_url = lenta_ru + item
    lenta_links.append(other_post_url)

for link in lenta_links:
    if 'moslenta' in link:
        news_obj = get_item_info_moslenta(link)
    else:
        news_obj = get_item_info_lenta(link)       

    # пополняем коллекцию
    obj = next(news.find({"url": {'$eq': news_obj['url']}}), None)
    if not obj:
        news.insert_one(news_obj)
    else:
        news.update_one({"id": {'$eq': news_obj['url']}},{'$set': news_obj})


In [10]:
db.news.count_documents({})

21

### Парсинг Яндекс.Новости

Принцип работы: 
1. Получаем список элементов (новостей) на странице (65 шт.)
2. Итерируемся по списку элементов и извлекаем и записываем в словарь ссылку на новость, заголовок, дату и время, название источника.
3. Для получения ссылки на источник переходим на саму новость (функция get_source_link_yandex написана для двух видов ссылок на случай, если на главной странице есть новость из Яндекс.Спорт)
4. Добавляем объекты в коллекцию news.

In [11]:
# функция для получения ссылки на источник с переходом на саму новость
def get_source_link_yandex(link):
    response = requests.get(link, headers=headers)
    
    if not response.ok:
        print(response.status_code)
        return
        
    root = html.fromstring(response.text)
    
    source_url = root.xpath("//h1[@class = 'story__head']//span[@class = 'story__head-agency']/a/@href")
    if source_url:
        source_url = source_url[0] 
    
    else:
        # ссылка на источник на яндекс спорт
        source_url = root.xpath("//div[@class = 'news-story__subtitle']/a/@href")[0]

    return source_url

In [12]:
yandex_page = requests.get(yandex_news, headers=headers)
root = html.fromstring(yandex_page.text) 

if not yandex_page.ok:
    print(yandex_page.status_code)
    
for item in root.xpath("//td[@class = 'stories-set__item']"):
    news_object = {}

    # ссылка
    link = item.xpath(".//h2[@class = 'story__title']/a/@href")
    if link:
        link = link[0]
        news_object['url'] = 'https://yandex.ru' + link
        # ссылка на источник (только в случае возможности получить ссылку на новость)
        news_object['source_url'] = get_source_link_yandex(news_object['url'])
    else:
        news_object['url'] = None

    # заголовок
    title = item.xpath(".//h2[@class = 'story__title']/a/text()")
    if title:
        news_object['title'] = title[0]
    else:
        news_object['title']  = None

    # дата, время и источник
    date_and_source = item.xpath(".//div[@class = 'story__date']/text()")
    
    # если элемент не найден - то ставим  current timestamp 
    if not date_and_source:
        news_object['date'] = str(datetime.datetime.now().date())
        news_object['time'] = str(datetime.datetime.now().time())
        news_object['source_title'] = None

    
    date_and_source = date_and_source[0]
    items = date_and_source.split(' ')
    
    # название источника
    news_object['source_title'] = ' '.join(items[:-1])

    # дата и время
    # сегодня (без указания даты)
    if '\xa0' not in items[-1]:
        news_object['time'] = items[-1] + ':00'
        news_object['date'] = str(datetime.date.today())  

    # вчера
    elif 'вчера' in items[-1]:
        time_complex = items[-1].split('\xa0')
        news_object['time'] = time_complex[-1] + ':00'
        news_object['date'] = str(datetime.date.today() - datetime.timedelta(1)) 

    # если указаны число и месяц (напр., 18 апреля в 22:41)
    else:
        time_complex = items[-1].split('\xa0')
        news_object['time'] = time_complex[-1] + ':00'

        day = time_complex[0]
        month = months[time_complex[1]]
        year = str(datetime.datetime.now().year)
        date = year + "-" + month + "-" + day
        news_object['date'] = date


    # пополняем коллекцию
    obj = next(news.find({"url": {'$eq': news_object['url']}}), None)
    if not obj:
        news.insert_one(news_object)
    else:
        news.update_one({"id": {'$eq': news_object['url']}},{'$set': news_object})

In [13]:
db.news.count_documents({})

85

In [17]:
df = pd.DataFrame(news.find({}))
df

,_id,url,title,source_title,source_url,date,time
0,5e9d6829b8a16c0325a84af3,https://news.mail.ru/society/41460513/,В России число заболевших коронавирусом превыс...,Интерфакс,http://www.interfax.ru/,2020-04-20,11:02:38
1,5e9d6829b8a16c0325a84af4,https://news.mail.ru/society/41458830/,Треть россиян поддерживают возможный перенос м...,Коммерсантъ,http://www.kommersant.ru,2020-04-20,09:58:57
2,5e9d6829b8a16c0325a84af5,https://news.mail.ru/society/41455831/,Названы российские регионы с приростом населения,РИА Новости,http://www.ria.ru,2020-04-20,00:42:37
3,5e9d6829b8a16c0325a84af6,https://news.mail.ru/incident/41456152/,У скончавшейся у подъезда москвички не выявлен...,Известия,http://iz.ru/,2020-04-20,01:40:13
4,5e9d682ab8a16c0325a84af7,https://news.mail.ru/incident/41458627/,"В МВД рассказали, что мошенники все чаще ворую...",ТАСС,http://www.tass.ru/,2020-04-20,10:21:06
...,...,...,...,...,...,...,...
80,5e9d6842b8a16c0325a84b43,https://yandex.ru/news/story/Mercedes-Benz_otz...,Mercedes-Benz отзывает около 500 автомобилей в...,32CARS.ru,https://www.32cars.ru/world-news/date-20-04-20...,2020-04-20,12:00:00
81,5e9d6842b8a16c0325a84b44,https://yandex.ru/news/story/Legendarnomu_VAZ-...,Легендарному ВАЗ-2101 исполнилось 50 лет,Автоновости дня,https://avtonovostidnya.ru/avtoprom/192112?utm...,2020-04-20,11:19:00
82,5e9d6842b8a16c0325a84b45,https://yandex.ru/news/story/Novyj_krossover_S...,Новый кроссовер Subaru получит имя Evoltis,Автоновости дня,https://avtonovostidnya.ru/novinki/192269-suba...,2020-04-20,11:58:00
83,5e9d6843b8a16c0325a84b46,https://yandex.ru/news/story/Haval_F7_stal_sam...,Haval F7 стал самым популярным китайским автом...,Автоновости дня,https://avtonovostidnya.ru/avtorynok/192119-ha...,2020-04-20,10:17:00


In [16]:
df.to_csv('news.csv')